In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import requests

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

# Model 1 (Provided Model)

In [ ]:
# Original Model Provided
def create_base_model(input_shape=(32, 32, 3), num_classes=len(class_names)):
    model = models.Sequential([
        # Convolutional layer 1
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        # Convolutional layer 2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        # Convolutional layer 3
        layers.Conv2D(64, (3, 3), activation='relu'),
        # Flattening the output for the dense layer
        layers.Flatten(),
        # Dense layer
        layers.Dense(64, activation='relu'),
        # Output layer
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
# Create and compile the base model
model1 = create_base_model()
model1.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
# Model summary
model1.summary()

history1 = model1.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history1.history['accuracy'], label='accuracy')
plt.plot(history1.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model1.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(f'test_loss: {test_loss}, test_accuracy: {test_acc}')

# Model 2 (Modified base on Provided Model)

In [ ]:
# A modified model base on the provided model
def create_base_modified_model(input_shape=(32, 32, 3), num_classes=len(class_names)):
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
# Create and compile the base model
model2 = create_base_modified_model()
model2.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
# Model summary
model2.summary()

history2 = model2.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history2.history['accuracy'], label='accuracy')
plt.plot(history2.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model2.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(f'test_loss: {test_loss}, test_accuracy: {test_acc}')

# Model 3 (Transfer Learning)

In [ ]:
def create_vgg16_model(input_shape=(32, 32, 3), num_classes=len(class_names)):
    inputs = tf.keras.Input(shape=input_shape)

    # Resize images to 224x224 within the model
    resized_inputs = tf.image.resize(inputs, (224, 224))

    # Preprocess the inputs as per VGG16's requirements
    x = preprocess_input(resized_inputs)

    # Load the VGG16 base model, excluding its top fully connected layers
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=x)

    # Freeze the layers of the base model to prevent them from being updated during training
    base_model.trainable = False

    # Extend the base model for multi-class classification
    x = layers.Flatten()(base_model.output)  # Flatten the output of the base model
    x = layers.Dense(512, activation='relu')(x)  # Add a fully connected layer
    x = layers.Dropout(0.5)(x)  # Incorporate dropout for regularization
    outputs = layers.Dense(num_classes, activation='softmax')(x)  # Final layer for multi-class classification

    # Construct the new model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
# Create and compile the base model
model3 = create_vgg16_model()
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Model summary
model3.summary()

history3 = model3.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history3.history['accuracy'], label='accuracy')
plt.plot(history3.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model3.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(f'test_loss: {test_loss}, test_accuracy: {test_acc}')

### Prediction the model

In [ ]:
# Function to load and preprocess an image
def load_and_preprocess_image(url, target_size=(32, 32)):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.resize(target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0
    return img

models = [model1, model2, model3]
def predict(model):
  predictions = model.predict(img)
  predicted_class = class_names[np.argmax(predictions[0])]
  print(f"Image URL: {url}")
  print(f"Predicted class: {predicted_class}\n")

# URLs of the images
image_urls = [
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0KnNu46povUcwKNf8fGiljnFfnD20nwYHd1r44JUikA&s",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSu-2Z3an9txHQSExt053UQzZEHkW_G0HZoXy_k1uQL2A&s",
    "https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSO7S6Lztyh63gN6oo1GESyOAfBwFYNkcGKBkrJlAgfkAm8Wa6PaQrQs233QFxBYzYn3Jy6g1AT5wwNl1X789grdblM_3JpruTLerJ5CA",
    "https://media.wired.com/photos/590951f9d8c8646f38eef333/4:3/w_929,h_697,c_limit/walmart-advanced-vehicle-experience-wave-concept-truck.jpg",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR49UeOkEvtCskTbXc5OQ4UTs3wTcaRUnRNjFO32FROCg&s",
    "https://i.natgeofe.com/k/520e971d-7a22-4a6f-90dc-258df74e45bc/american-goldfinch.jpg"
]

# Load, preprocess, and predict each image
for url in image_urls:
    img = load_and_preprocess_image(url)
    for i, m in enumerate(models):
      print(f'Model: {i+1}')
      predict(m)


In [ ]:
# Function to load and preprocess an image
def load_and_preprocess_image(url, target_size=(32, 32)):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.resize(target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0
    return img

def predict_and_show(model, img, url):
    predictions = model.predict(img)
    predicted_class = class_names[np.argmax(predictions[0])]
    plt.imshow(img[0])
    plt.title(f"Predicted class: {predicted_class}")
    plt.axis('off')
    plt.show()
    print(f"Image URL: {url}")
    print(f"Predicted class: {predicted_class}\n")

models = [model1, model2, model3]

# URLs of the images
image_urls = [
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0KnNu46povUcwKNf8fGiljnFfnD20nwYHd1r44JUikA&s",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSu-2Z3an9txHQSExt053UQzZEHkW_G0HZoXy_k1uQL2A&s",
    "https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSO7S6Lztyh63gN6oo1GESyOAfBwFYNkcGKBkrJlAgfkAm8Wa6PaQrQs233QFxBYzYn3Jy6g1AT5wwNl1X789grdblM_3JpruTLerJ5CA",
    "https://media.wired.com/photos/590951f9d8c8646f38eef333/4:3/w_929,h_697,c_limit/walmart-advanced-vehicle-experience-wave-concept-truck.jpg",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR49UeOkEvtCskTbXc5OQ4UTs3wTcaRUnRNjFO32FROCg&s",
    "https://i.natgeofe.com/k/520e971d-7a22-4a6f-90dc-258df74e45bc/american-goldfinch.jpg"
]

# Load, preprocess, and predict each image
for url in image_urls:
    img = load_and_preprocess_image(url)
    for i, m in enumerate(models):
      print(f'Model: {i+1}')
      predict_and_show(m, img, url)
